
we will take our projects from Day 1 
Business Challenge:
Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.
We will be provided a company name and their primary website.

In [70]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown,display,update_display
from openai import OpenAI

In [71]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key[:8] == 'sk-proj-':
    print("API Key looks good")
else:
    print("Issue getting API Key")

MODEL = 'gpt-4o-mini'
openai = OpenAI()

API Key looks good


In [72]:
# Class represent webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """Class to represent website that we have scraped , now with links"""

    def __init__(self,url):
        self.url = url
        response = requests.get(url,headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body,'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script","style","img","input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n",strip=True)
        else:
            self.text = ""
        
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]
    
    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"
        
        

In [73]:
ws = Website("https://huggingface.co/")
ws.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/ByteDance-Seed/BAGEL-7B-MoT',
 '/mistralai/Devstral-Small-2505',
 '/google/gemma-3n-E4B-it-litert-preview',
 '/google/medgemma-4b-it',
 '/ByteDance/Dolphin',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/Lightricks/ltx-video-distilled',
 '/spaces/NihalGazi/FLUX-Pro-Unlimited',
 '/spaces/stepfun-ai/Step1X-3D',
 '/spaces/NihalGazi/Text-To-Speech-Unlimited',
 '/spaces',
 '/datasets/disco-eth/EuroSpeech',
 '/datasets/openbmb/Ultra-FineWeb',
 '/datasets/ministere-culture/comparia-conversations',
 '/datasets/nvidia/OpenCodeReasoning',
 '/datasets/nvidia/OpenMathReasoning',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Wr

Go with GPT-4o-mini figure out which links are relevant
Use a call to gpt-4o-mini to read the links on a webpage, and response in structured JSON.
It decide which links are relevant, and replace relative links such "/about" with "https://company.com/about".

This is good use case for an LLM. Without this you need to implement very complax logic to get parsed links , It would be very hard.


In [74]:
link_system_prompt = "You are provided with a list of links which is found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [75]:
print(link_system_prompt)

You are provided with a list of links which is found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [76]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
    Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [77]:
print(get_links_user_prompt(ws))

Here is the list of links on the website of https://huggingface.co/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format.     Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/
/models
/datasets
/spaces
/docs
/enterprise
/pricing
/login
/join
/spaces
/models
/ByteDance-Seed/BAGEL-7B-MoT
/mistralai/Devstral-Small-2505
/google/gemma-3n-E4B-it-litert-preview
/google/medgemma-4b-it
/ByteDance/Dolphin
/models
/spaces/enzostvs/deepsite
/spaces/Lightricks/ltx-video-distilled
/spaces/NihalGazi/FLUX-Pro-Unlimited
/spaces/stepfun-ai/Step1X-3D
/spaces/NihalGazi/Text-To-Speech-Unlimited
/spaces
/datasets/disco-eth/EuroSpeech
/datasets/openbmb/Ultra-FineWeb
/datasets/ministere-culture/comparia-conversations
/datasets/nvidia/OpenCodeReasoning
/datasets/nvidia/OpenMathReasoning
/datasets
/join
/pricing#endpoints
/pricing#spaces
/pricing
/enterprise
/enterprise
/enterprise
/ent

In [78]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = MODEL,
        messages = [
            {"role":"system","content":link_system_prompt},
            {"role":"user","content":get_links_user_prompt(website)}
        ],
        response_format={"type":"json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [79]:
huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/ByteDance-Seed/BAGEL-7B-MoT',
 '/mistralai/Devstral-Small-2505',
 '/google/gemma-3n-E4B-it-litert-preview',
 '/google/medgemma-4b-it',
 '/ByteDance/Dolphin',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/Lightricks/ltx-video-distilled',
 '/spaces/NihalGazi/FLUX-Pro-Unlimited',
 '/spaces/stepfun-ai/Step1X-3D',
 '/spaces/NihalGazi/Text-To-Speech-Unlimited',
 '/spaces',
 '/datasets/disco-eth/EuroSpeech',
 '/datasets/openbmb/Ultra-FineWeb',
 '/datasets/ministere-culture/comparia-conversations',
 '/datasets/nvidia/OpenCodeReasoning',
 '/datasets/nvidia/OpenMathReasoning',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Wr

In [80]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community discussion page',
   'url': 'https://discuss.huggingface.co'},
  {'type': 'status page', 'url': 'https://status.huggingface.co/'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

Now Make the brochure ....
Assemble all the details into another prompt to GPT4-o

In [81]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result



In [82]:
print(get_all_details("https://huggingface.co"))

Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
ByteDance-Seed/BAGEL-7B-MoT
Updated
4 days ago
•
3.09k
•
648
mistralai/Devstral-Small-2505
Updated
about 2 hours ago
•
82.8k
•
551
google/gemma-3n-E4B-it-litert-preview
Updated
about 4 hours ago
•
492
google/medgemma-4b-it
Updated
5 days ago
•
12.6k
•
231
ByteDance/Dolphin
Updated
4 days ago
•
1.25k
•
173
Browse 1M+ models
Spaces
Running
7.21k
7.21k
DeepSite
🐳
Generate any application with DeepSeek
Running
on
Zero
530
530
LTX Video Fast
🎥
ultra-fast video model, LTX 0.9.7 13B distilled
Running
709
709
FLUX Pro Unlimited
🔥
Use the FLUX-Pro model as much as you want.
Running
on
Zero
196
196
Step1X 3D
🐨
i

In [83]:

system_prompt = "You are an assistant tasked with analyzing key pages from a company's website and and creates a short concise brochure for potential customers,\
investors, and job seekers. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

In [84]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [85]:
def create_brochure(company_name,url):
    response = openai.chat.completions.create(
        model = MODEL,
        messages = [
            {"role":"system","content" : system_prompt},
            {"role":"user","content" : get_brochure_user_prompt(company_name,url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [86]:
create_brochure("HuggingFace","https://huggingface.co")

# Hugging Face Brochure

---

## Welcome to Hugging Face

**The AI community building the future!**  
At Hugging Face, we provide a collaborative platform where the machine learning community can explore, develop, and share models, datasets, and applications.

### Key Features

- **1M+ Models**: Discover and leverage a vast library of machine learning models.
- **Datasets**: Access over 250,000 datasets for your machine learning needs.
- **Spaces**: Create and showcase applications with ease.
- **Enterprise Solutions**: Advanced support and security for organizations looking to harness AI effectively.

---

### Customer Focus

Hugging Face supports over **50,000 organizations**, including industry giants such as:

- Amazon
- Google
- Microsoft
- Intel
- Meta

By joining us, businesses of all sizes can leverage cutting-edge AI tools and innovations.

---

### Company Culture

At Hugging Face, we foster a vibrant, collaborative environment that values creativity and open-source contributions. Our community is passionate about pushing the boundaries of AI and machine learning, and we believe in building tools that anyone can access and contribute to.

- **Open Source Commitment**: We invite and encourage contributions from the community, ensuring our tools remain at the forefront of AI advancements.
- **Collaboration**: We emphasize teamwork and shared knowledge to achieve common goals.

---

### Careers at Hugging Face

Join our dynamic team! We seek innovative minds who are excited about shaping the future of AI. Positions cover various aspects, from engineering to community management.

#### Current Openings
- AI Researchers
- Software Engineers
- Community Managers
- Product Designers

Explore available opportunities and grow your career in the heart of the AI revolution.

---

### Join Us

Whether you are a potential customer looking to enhance your organization’s capabilities, an investor interested in the AI vision, or a job seeker eager to be part of this exciting journey, **Hugging Face** welcomes you. 

Visit us at [Hugging Face](https://huggingface.co) to explore more!

--- 

*Together, let’s build the future of AI!*

Lets Make OpenAI output response in streaming form like typewriter animation

In [90]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [92]:
stream_brochure("Huggingface","https://huggingface.co")

# Hugging Face Brochure

## Welcome to Hugging Face
Hugging Face is an innovative platform at the forefront of the AI community, dedicated to building the future of machine learning. We provide a collaborative environment where developers, researchers, and enthusiasts converge to create and explore state-of-the-art models, datasets, and applications.

## Our Offerings

### **Powerful Tools for Collaboration**
- **Models:** Access a vast library of **1M+ models**, including the latest advancements from industry leaders.
- **Datasets:** Choose from **250k+ datasets** tailored for various ML tasks, ensuring your projects have the best available resources.
- **Spaces:** Share and host your applications with unlimited public access, fostering community engagement and collaboration.

### **Enterprise Solutions**
For organizations seeking cutting-edge tools, we offer:
- Tailored enterprise-grade services featuring **dedicated support**, **Single Sign-On**, and **advanced security**.
- Cost-effective compute solutions, starting from **$0.60/hour** for optimized GPU capabilities.

## Community and Collaboration
Hugging Face is not just a platform; it is a **thriving community**. With over **50,000 organizations** leveraging our resources—ranging from non-profits to high-profile tech giants like Google and Microsoft—our open-source tools are empowering ML advancements worldwide.

### Key Collaborators Include:
- **Amazon**
- **Meta**
- **Intel**
- **Grammarly**

## Company Culture
At Hugging Face, we believe in fostering a culture of **collaboration, innovation, and creativity**. We aim to build an inclusive environment where everyone can contribute and grow. Our community-driven approach allows team members to learn from each other and share insights transparently.

### **Join Our Team**
We're always looking for talented individuals who are passionate about AI and want to make a difference. Explore our careers page to discover job openings that match your skills and join a team that is supportive and driven by a shared mission to advance machine learning.

## Connect with Us
Whether you're a potential customer, investor, or job seeker, we invite you to be part of the journey at Hugging Face. 

- [Learn More](https://huggingface.co)
- [Join Our Community](https://huggingface.co/community)
- [Careers](https://huggingface.co/jobs)

Together, let's build a better future powered by AI!